In [333]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import BayesianEstimator
from pgmpy.models import NaiveBayes

In [334]:
tennis_dataset = pd.read_csv('D:/MSC/UHull/COURSE/WEEK04-PROBABILISTIC REASONING/CHALLENGE ACTIVITY/tennis_data.txt',header = None)
tennis_dataset = pd.DataFrame(tennis_dataset)
tennis_dataset.columns = ['Day','Outlook','Temperature','Humidity','Wind','PlayTennis']

In [335]:
tennis_dataset

,Day,Outlook,Temperature,Humidity,Wind,PlayTennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [336]:
model_tennis = BayesianNetwork([('PlayTennis','Outlook'),('PlayTennis','Temperature'),('PlayTennis','Humidity'),('PlayTennis','Wind')])
model_tennis.fit(tennis_dataset,estimator=BayesianEstimator,prior_type='BDeu', equivalent_sample_size=1)
for cpd in model_tennis.get_cpds():
    print(cpd)

+-----------------+----------+
| PlayTennis(No)  | 0.366667 |
+-----------------+----------+
| PlayTennis(Yes) | 0.633333 |
+-----------------+----------+
+-------------------+--------------------+---------------------+
| PlayTennis        | PlayTennis(No)     | PlayTennis(Yes)     |
+-------------------+--------------------+---------------------+
| Outlook(Overcast) | 0.0303030303030303 | 0.4385964912280702  |
+-------------------+--------------------+---------------------+
| Outlook(Rain)     | 0.3939393939393939 | 0.3333333333333333  |
+-------------------+--------------------+---------------------+
| Outlook(Sunny)    | 0.5757575757575757 | 0.22807017543859648 |
+-------------------+--------------------+---------------------+
+-------------------+---------------------+---------------------+
| PlayTennis        | PlayTennis(No)      | PlayTennis(Yes)     |
+-------------------+---------------------+---------------------+
| Temperature(Cool) | 0.21212121212121213 | 0.3333333333333333

In [337]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model_tennis)

In [338]:
test = infer.query(variables = {'Outlook' : 'Overcast','Wind' : 'Weak','Temperature' : 'Mild', 'Humidity' : 'Normal'}, 
                   # evidence={'PlayTennis' : 'Yes','Temperature' : 'Hot'})
                   evidence={'PlayTennis' : 'No'})
print(test)

+-------------------+--------------+-------------------+------------------+------------------------------------------+
| Outlook           | Wind         | Temperature       | Humidity         |   phi(Outlook,Wind,Temperature,Humidity) |
+===================+==============+===================+==================+==========================================+
| Outlook(Overcast) | Wind(Strong) | Temperature(Cool) | Humidity(High)   |                                   0.0029 |
+-------------------+--------------+-------------------+------------------+------------------------------------------+
| Outlook(Overcast) | Wind(Strong) | Temperature(Cool) | Humidity(Normal) |                                   0.0009 |
+-------------------+--------------+-------------------+------------------+------------------------------------------+
| Outlook(Overcast) | Wind(Strong) | Temperature(Hot)  | Humidity(High)   |                                   0.0055 |
+-------------------+--------------+------------